In [104]:
import numpy as np
import sympy as sy
import scipy as spy
from scipy.linalg import expm
import pylab as py 
from random import random

In [2]:
def normalize(M):
    """
    Subfunction for T. It normalizes the matrix given as input.
    
    INPUT:
        - M = A matrix M.
        
    OUTPUT:
        - M0 = The matrix M normalized, with rows that add to 1.     
    """
    
    M0=np.array(M)
    if M0.ndim == 1:
        s= M0.sum()
        return np.divide(M0,s)
        
    elif M0.ndim == 2:
        s=M0.sum(axis=1)
        return np.divide(M0,s[:,np.newaxis])
    else:
        return "Normalize. Wrong input"

In [4]:
def stat(T):
    w, v = np.linalg.eig(T.T)
    
    j_stat =np.argmin(abs(w-1.0))
    p_stat=v[:,j_stat].real
    p_stat /= p_stat.sum()
    return p_stat

In [57]:
T=np.matrix([[0.9,0.1,0,0],[0.1,0.89,0.01,0],[0,0.01,0.79,0.2],[0,0,0.2,0.8]])
stat(T)

matrix([[ 0.25],
        [ 0.25],
        [ 0.25],
        [ 0.25]])

In [7]:
def mean_recurrence_time(T):
    """
    The mean_recurrence_time (μ_i) is the mean time to return to state i, given that we start in state i. 
    In other words, Let T_i be the first time we enter state i, starting from time 1. So we have that
    μ_i = E[T_i | X_0 = i]. μ_i is the inverse of the stationary distribution.
    
    INPUT:
        - T = A markov transition matrix.
        
    OUTPUT:
        - A vector. The inverse of the stationary distribution.
    """
    
    pi=stationary_distribution(T)
    mr=np.ones(len(pi))
    return np.divide(mr,pi)

In [105]:
def implied_timescales(T,lag):
    """
    Calculates timescale_i(k*lag)=
    
    INPUT:
        - T = A markov transition matrix.
        - lag = reference lagtime we are considering.
    """
    
    
    eig_val, eig_vec=np.linalg.eig(expm(T,lag))
    L=[]
    for i in range(len(eig_val)):
        L.append(-(lag)/(np.log(np.absolute(eig_val[i]))))
    return(L)

In [106]:
implied_timescales(T,3)

[-3.7741460861436424,
 -3.0000000000000022,
 -3.0291524798495226,
 -5.0442025019265255]

In [43]:
s=np.array([[0.8,0.1,0.1,0,0,0,0,0,0],[0.1,0.8,0.1,0,0,0,0,0,0],[0.1,0.1,0.75,0.05,0,0,0,0,0],
           [0,0,0.05,0.75,0.1,0.1,0,0,0],[0,0,0,0.1,0.8,0.1,0,0,0],[0,0,0,0.1,0.1,0.795,0.005,0,0],
           [0,0,0,0,0,0.005,0.795,0.1,0.1],[0,0,0,0,0,0,0.1,0.8,0.1],[0,0,0,0,0,0,0.1,0.1,0.8]])
implied_timescales(s,1)

[35.403688032812397,
 1501199875790165.8,
 424.31824323124607,
 2.1385733837202801,
 2.7343778941652412,
 2.8036732520571248,
 2.803673252057131,
 2.8036732520571297,
 2.8036732520571297]

In [10]:
def hitting_time_ij(T,i,j,steps):
    """
    Calculates the hitting time between states i, j from the markov transition matrix.
    Since p_ij is the probability of going to state j given that we are in state 
    i, the product π_i * p_ij is the long run proportion of transitions that go from state
    i to state j.
    
    INPUT:
        - T = A markov transiton matrix.
        - i = Some state from which we start.
        - j = Some state where we wanna arrive.
        - steps = An integer. Max number of steps we want to consider.
        
    OUTPUT:
        - A picture showing how hitting probability between i and j changes with time.
        - Limiting probability of going from state i to state j.

    """
    
    A=T.copy()
    
    hit_idx = (i,j)

    # Make the final state an absorbing condition
    A[hit_idx[1],:] = 0
    A[hit_idx[1],hit_idx[1]] = 1

    # Make a proper Markov matrix by row normalizing
    A = (A.T/A.sum(axis=1)).T
    
    B = A.copy()
    Z = []
    for n in range(steps):
        Z.append( B[hit_idx] )
        B = py.dot(B,A)
    
    py.plot(Z)
    py.xlabel("steps")
    py.ylabel("hit probability")
    py.show()

In [11]:
hitting_time_ij(T,0,3,2000)

In [12]:
s=np.array([[0.8,0.1,0.1,0,0,0,0,0,0],[0.1,0.8,0.1,0,0,0,0,0,0],[0.1,0.1,0.75,0.05,0,0,0,0,0],
           [0,0,0.05,0.75,0.1,0.1,0,0,0],[0,0,0,0.1,0.8,0.1,0,0,0],[0,0,0,0.1,0.1,0.795,0.005,0,0],
           [0,0,0,0,0,0.005,0.795,0.1,0.1],[0,0,0,0,0,0,0.1,0.8,0.1],[0,0,0,0,0,0,0.1,0.1,0.8]])
stat(s)

array([ 0.11111111,  0.11111111,  0.11111111,  0.11111111,  0.11111111,
        0.11111111,  0.11111111,  0.11111111,  0.11111111])

In [13]:
hitting_time_ij(s,0,2,2000)

In [14]:
T

matrix([[ 0.9  ,  0.1  ,  0.   ,  0.   ],
        [ 0.1  ,  0.895,  0.005,  0.   ],
        [ 0.   ,  0.005,  0.795,  0.2  ],
        [ 0.   ,  0.   ,  0.2  ,  0.8  ]])

In [90]:
def plot_timescales(T,LAG):
    """
    
    INPUT:
        - n_eig: number of eigenvalues to be ploted. The first n_eig eigenvalues will be ploted.
    """
    
    eig=np.linalg.eig(T)
    k=len(eig[0])
    
    d=len(T)
    t=[]
    
    for i in range(k):
        t.append([])
        
    for lag in LAG:
        t_lag=implied_timescales(T,lag)
        for j in range(k):
            if -100<t_lag[j]<100:
                (t[j]).append(t_lag[j])
            else:
                (t[j]).append(0)
    for i in range(len(t)):
        py.plot(LAG,t[i],'o',linestyle='-')
    py.xlabel('Lag time')
    py.ylabel('Implied timescale')
    py.show()    

In [107]:
LAG=np.array([1,2,3,4,5,7,10,15,20,30,40,50,60,80,100])
plot_timescales(T,LAG)

In [34]:
s

array([[ 0.8  ,  0.1  ,  0.1  ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.1  ,  0.8  ,  0.1  ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.1  ,  0.1  ,  0.75 ,  0.05 ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.05 ,  0.75 ,  0.1  ,  0.1  ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.1  ,  0.8  ,  0.1  ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.1  ,  0.1  ,  0.795,  0.005,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.005,  0.795,  0.1  ,
         0.1  ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.1  ,  0.8  ,
         0.1  ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.1  ,  0.1  ,
         0.8  ]])

In [84]:
LAG=np.array([1,2,3,4,5,7,10,15,20,30,40,50,60,80,100])
plot_timescales(s,LAG)

In [85]:
LAG2=np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
plot_timescales(s,LAG2)

In [91]:
plot_timescales(T,LAG2)

In [50]:
LAG3=np.array([1,5,10,15,20,30,50,70,100,200,300,400,500,700,1000,1300,1700,2000])

In [ ]:
def decay_timescale(T):
    """
    It computes the decay or relaxation timescale of an ergodic Markov chain. In such a chain,
    the amplitudes of the distortions from equilibrium decay exponentially with discrete time k.
    
    INPUT:
        -
        
    OUTPUT:
        - 
    """
    
    eig=np.linalg.eig(T)
    eig=eig[0]
    eig=np.absolute(eig)
    t=np.ones(len(eig))
    return np.divide(t,(-1*(np.log(eig))))

In [ ]:
def paso(i,T):
    """
    Function that simulates a random pass from state i in a markov model. It gives back the vertex
    that is reached.
    
    INPUT:
        - i: the original vertex.
        - T: the Markov transition matrix.
        
    OUTPUT:
        - j: the vertex reached from i in this random experiment.
    """

    T=np.squeeze(np.asarray(T))
    Fi=[sum(T[i][:j+1]) for j in range(len(T[i]))]
    a=random()
    k=0
    while a>=Fi[k]:
        k+=1
    return k

In [ ]:
"Test to check whether paso works."

m=1000

vertex=[0,1,2,3]
L=[[],[],[],[]]

for i in range(m):
    for v in vertex:
        L[v].append(paso(v,T))
for v in vertex:
    print((L[v].count(0))/m,(L[v].count(1))/m,(L[v].count(2))/m,(L[v].count(3))/m)

In [ ]:
def camino(i,pasos,T):
    estado=[i]
    for k in range(pasos):
        estado.append(paso(estado[-1],T))
    return estado

In [ ]:
viaje=camino(2,100,T)
viaje.count(0),viaje.count(1),viaje.count(2),viaje.count(3)

In [ ]:
"""Test to check wheter in long trips from random initial vertices we end up in a vertex with the prob
of the stationary distribution"""

m1=1000
m2=100

vertices=[0,1,2,3]
L=[[],[],[],[]]

for i in range(m1):
    a=random()
    if 0<a<=0.25:
        l=0
    elif 0.25<a<=0.5:
        l=1
    elif 0.5<a<=0.75:
        l=2
    elif 0.75<a<=1:
        l=3
    viaje=camino(l,m2,T)
    L[l].append(viaje[len(viaje)-1])
for v in vertex:
    print((L[v].count(0))/(len(L[v])),(L[v].count(1))/(len(L[v])),(L[v].count(2))/(len(L[v])),
          (L[v].count(3))/(len(L[v])))

In [ ]:
P=np.matrix([[1/4,3/4],[1/5,4/5]])

"""Test to check the medium time we stay in each state when we run m1 paths of m2 steps"""

m1=1000
m2=100

vertices=[0,1]
L=[[],[]]

for i in range(m1):
    a=random()
    if 0<a<=0.5:
        l=0
    elif 0.5<a<=1:
        l=1
    viaje=camino(l,m2,P)
    for l in vertices:
        L[l].append(viaje.count(l))
for v in vertices:
    print((np.array(L[v])).sum())

In [ ]:
T=np.matrix([[0.9,0.1,0,0],[0.1,0.895,0.005,0],[0,0.005,0.795,0.2],[0,0,0.2,0.8]])

"""Test to check the medium time we stay in each state when we run m1 paths of m2 steps"""

m1=1000
m2=100

vertices=[0,1,2,3]
L=[[],[],[],[]]

for i in range(m1):
    a=random()
    if 0<a<=0.25:
        l=0
    elif 0.25<a<=0.5:
        l=1
    elif 0.5<a<=0.75:
        l=2
    elif 0.75<a<=1:
        l=3
    viaje=camino(l,m2,T)
    for l in vertices:
        L[l].append(viaje.count(l))
for v in vertices:
    print((np.array(L[v])).sum())